In [1]:


import os 
from dotenv import load_dotenv
load_dotenv()
valueENV = os.getenv("DBconnection") 

from sqlalchemy import create_engine
import pandas as pd
engine = create_engine(valueENV)
df = pd.read_sql_query("select sum(total_price) as total_price,branch_id,employee_id from transaction group by branch_id,employee_id", engine)


In [4]:
df['branch_id'].unique()

array([1, 2, 3], dtype=int64)

In [5]:
df['branch_id'].unique()[1]

2

In [2]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

DATABASE_URL=os.getenv("DBconnection")
load_dotenv()
db = SQLDatabase.from_uri(DATABASE_URL)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 30 employees.'

In [8]:
b = chain.invoke({"question": "what is the highest sale product for each branch"})

In [9]:
print(b)

The highest sale product for the Central Branch is the Stool with a sale amount of 1500.


In [10]:
c = chain.invoke({"question": "what is the lowest sale product for each branch and what's best stetragy to boost it"})
print(c)

The lowest sale product for each branch based on the SQL query and result provided are as follows:
1. Central Branch - Apple with a lowest sale price of 13
2. Central Branch - Rice with a lowest sale price of 14
3. Central Branch - Soda with a lowest sale price of 15
4. Central Branch - Egg with a lowest sale price of 20
5. Central Branch - Meat with a lowest sale price of 30

To boost the sales of the lowest sale product for each branch, a possible strategy could be to run promotions or discounts specifically targeting those products. This could include offering bundle deals, discounts for purchasing in bulk, or running targeted marketing campaigns to increase awareness and demand for those products. Additionally, analyzing customer feedback and preferences for those products could help in tailoring marketing strategies to better appeal to the target audience.


In [11]:
c = chain.invoke({"question": "what is the lowest sale product for every branch and what's best stetragy to boost it"})
print(c)

The lowest sale product for every branch is as follows:
- Central_Branch: Milk with a sale price of 30
- South_Branch: Apple with a sale price of 39
- Nowhere_Branch: Rice with a sale price of 112

To boost the sales of these products, some strategies could include:
- Offering discounts or promotions on these products to attract customers
- Improving the visibility and placement of these products in the store
- Conducting targeted marketing campaigns to promote these products to the target audience
- Gathering customer feedback and making improvements to the product based on their preferences and needs
- Collaborating with suppliers to negotiate better prices for these products, allowing for more competitive pricing.


In [11]:
print(repr(c))

'The lowest sale product for each branch based on the SQL query and result provided are as follows:\n1. Central Branch - Apple with a lowest sale price of 13\n2. Central Branch - Rice with a lowest sale price of 14\n3. Central Branch - Soda with a lowest sale price of 15\n4. Central Branch - Egg with a lowest sale price of 20\n5. Central Branch - Meat with a lowest sale price of 30\n\nTo boost the sales of the lowest sale product for each branch, a possible strategy could be to run promotions or discounts specifically targeting those products. This could include offering bundle deals, discounts for purchasing in bulk, or running targeted marketing campaigns to increase awareness and demand for those products. Additionally, analyzing customer feedback and preferences for those products could help in tailoring marketing strategies to better appeal to the target audience.'


In [12]:
c = chain.invoke({"question": "what is the lowest total sale product for every branch and what's best stetragy to boost it"})
print(c)

The lowest total sale product for every branch is as follows:
- South_Branch: Apple with a total sale of 13
- Nowhere_Branch: Apple with a total sale of 13
- Central_Branch: Apple with a total sale of 13
- Central_Branch: Rice with a total sale of 14
- Nowhere_Branch: Rice with a total sale of 14

To boost the lowest total sale product, the best strategy would depend on various factors such as the target market, competition, and product characteristics. However, some general strategies to consider could be:
1. Analyze the reasons behind the low sales and identify any potential issues or barriers.
2. Conduct market research to understand customer preferences and needs.
3. Improve the product's features, quality, or packaging to make it more appealing to customers.
4. Offer promotions or discounts to incentivize customers to purchase the product.
5. Increase marketing efforts to raise awareness and visibility of the product.
6. Collaborate with influencers or industry experts to endorse 

In [35]:
df = pd.read_sql_query("""
select 
a.id,
a.total_price,
-- a.employee_id,
-- a.cust_id,
date(a.tran_date) as tran_date,
-- b.id,
b.price,
b.amount,
b.amount_price as order_price,
b.product_id,
b.tran_id,
c.id as branch_id,
c.name as branch_name,
c.address_province as branch_address,
-- d.id,
d.name as product_cat,
-- e.id,
e.name as product_name,
e.unit_price,
e.cat_id

-- c.*,extract(month from a.tran_date) "tran_month" ,extract(year from a.tran_date) as "tran_year"
-- ,b.*
-- ,b.amount,b.price,b.amount_price,b.product_id,b.tran_id
from transaction a 
left join "order" b on a.id = b.tran_id
left join branch c on a.branch_id = c.id
left join categories d on b.product_id = d.id
left join product e on b.product_id = e.id
-- where 
order by a.id asc
""", engine)


In [36]:
distinct_dates = df['tran_date'].drop_duplicates()
distinct_dates = pd.to_datetime(distinct_dates, format='%Y-%m-%d').dt.year.drop_duplicates()


In [37]:
distinct_dates

0    2024
Name: tran_date, dtype: int32

In [15]:
# distinct_dates
marks={i: date.strftime('%Y-%m-%d') for i, date in enumerate(distinct_dates)},
        

In [16]:
marks


({0: '2023-03-10',
  1: '2023-05-18',
  2: '2023-03-09',
  3: '2023-03-03',
  4: '2022-04-19',
  5: '2021-04-01',
  6: '2022-04-16',
  7: '2023-03-23',
  8: '2021-06-21',
  9: '2022-03-24',
  10: '2022-09-28',
  11: '2023-07-05',
  12: '2021-10-29',
  13: '2021-10-01',
  14: '2021-08-20',
  15: '2023-06-22',
  16: '2022-10-09',
  17: '2022-10-23',
  18: '2022-01-30',
  19: '2021-04-24',
  20: '2020-08-19',
  21: '2020-10-11',
  22: '2022-12-15',
  23: '2023-10-05',
  24: '2022-01-21',
  25: '2022-11-26',
  26: '2023-08-03',
  27: '2023-06-30',
  28: '2022-08-12',
  29: '2022-07-15',
  30: '2023-06-18',
  31: '2022-04-30',
  32: '2021-02-23',
  33: '2023-04-04',
  34: '2023-11-25',
  35: '2022-01-24',
  36: '2022-07-29',
  37: '2021-04-30',
  38: '2021-08-12',
  39: '2021-12-25',
  40: '2023-09-21',
  41: '2021-02-10',
  42: '2020-11-29',
  43: '2023-12-24',
  44: '2022-08-06',
  45: '2023-05-19',
  46: '2023-10-21',
  47: '2021-06-29',
  48: '2022-04-02',
  49: '2023-06-14',
  50: '202